## PhysionetMI

Load data of all 109 subjects.

Only the runs of "hands vs feet" motor imagery are loaded [runs 6, 10, 14]. Data is extracted as mne.io.Raw objects, each object representing the concatenated runs of a single subject.

In [ ]:
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.channels import make_standard_montage
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.channels import make_standard_montage
from sklearn.pipeline import Pipeline
from scripts.raw_preprocessing import FilterRaw, RemoveArtifacts, SelectChannels, Epochify

runs = [6, 10, 14]  # motor imagery: hands vs feet
montage = make_standard_montage("standard_1005")

channels = ["C3", "C4", "P3", "P4", "T7", "T8", "P7", "P8"] # These positions of the 10-10 system are equivalent to our positions in the 10-20 system
event_ids = ['rest', 'hands', 'feet']

physionet_preprocessing_pipeline = Pipeline([
    ("filter", FilterRaw()),
    ("remove_artifacts", RemoveArtifacts()),
    # ("select_channels", SelectChannels(channels=channels)),
    ("epochify", Epochify(event_ids=event_ids, channels=channels)),
])

physionet_subjects = []

for subject in range(1, 110):
    f_names = eegbci.load_data(subject, runs=runs, path="datasets", update_path=True)
    raw = concatenate_raws([read_raw_edf(f_name, preload=True) for f_name in f_names])

    if raw.info["sfreq"] == 160:
        eegbci.standardize(raw)  # set channel names
        raw.set_montage(montage)
        raw.annotations.rename(dict(T0="rest", T1="hands", T2="feet"))
        raw.set_eeg_reference(projection=True)

        subject_epochs = physionet_preprocessing_pipeline.fit_transform(raw)

        physionet_subjects.append(subject_epochs)

# Save the preprocessed data in /data/physionet.pkl
import pickle
with open("pickled_data/physionet.pkl", "wb") as f:
    pickle.dump(physionet_subjects, f)

## Schirrmeister 2017

In [ ]:
from moabb.datasets import Schirrmeister2017

dataset = Schirrmeister2017()
sessions = dataset.get_data()